# Sesión 2: Modelos que aprenden farmocología

En esta sesión utilizaremos `smina` y `diffdock`para realizar el docking sobre una GPCR (PDB id:6A93)


*   Redocking: Son capaces los modelos de replicar los datos experimentales (cristal)
*   Docking con 5 ligandos: Vamos a ver según los modelos cuál es mas probable que se una con mayor afinidad.

## Redocking
### Docking clasico `smina`

Primero vamos a instalar el programa



In [ ]:
# @title Instalar smina
# Instalando y cargando dependencias

!pip install -q condacolab --quiet
import condacolab
condacolab.install()
!conda install conda-forge::openbabel -y --quiet
!pip install py3dmol --quiet # 3D Molecular Visualizer
!pip install rdkit --quiet
!pip install biopython --quiet
!pip install MDAnalysis --quiet
!pip install pandas --quiet
# Instalando smina

!wget https://sourceforge.net/projects/smina/files/smina.static/download -O smina.static
!chmod u+x smina.static
!./smina.static

Descargamos el cristal del PDB.

In [ ]:
!wget http://files.rcsb.org/download/6A93.pdb

In [ ]:
!grep "ATOM.* A " 6A93.pdb > rec.pdb # Extraemos la proteina
!grep "8NU A" 6A93.pdb > lig.pdb # Extraemos el ligando

In [ ]:
!obabel rec.pdb -O rec.pdb -p # Añadimos hidrogenos a la proteina a pH fisiologico 7.4
!obabel lig.pdb -O lig.pdb -p # Añadimos hidrogenos al ligando

Vamos a realizar el re-docking ahora con `smina`.

In [ ]:
!./smina.static -r rec.pdb -l lig.pdb --autobox_ligand lig.pdb --autobox_add 8 --exhaustiveness 16 -o redock.pdbqt  --scoring vina # vinardo, ad4_scoring, dkoes_fast, dkoes_scoring

In [ ]:
!obabel redock.pdbqt -opdb -O redock.pdb

Vamos a visualizar ahora los resultados.

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.12/site-packages/')
import py3Dmol

# Function to read PDB file
def read_pdb(inputs):
    with open(inputs) as ifile:
        system = "".join([x for x in ifile])
    return system

# Load protein and reference ligand from PDB files
protein_file = 'rec.pdb'
reference_ligand_file = 'lig.pdb'
dock_file = 'redock.pdb'

protein = read_pdb(protein_file)
reference = read_pdb(reference_ligand_file)
ligands = read_pdb(dock_file)

# Initialize py3Dmol viewer
viewer = py3Dmol.view(width=800, height=600)

# Add protein to viewer
viewer.addModel(protein, 'pdb')
viewer.setStyle({'model': 0}, {'cartoon': {'color': 'spectrum'}})

# Add reference ligand to viewer
viewer.addModel(reference, 'pdb')
viewer.setStyle({'model': 1}, {'stick': {'color': 'red'}})

# Function to add ligand poses dynamically
def add_ligand_pose(viewer, ligands, pose_number):
    ligand_models = ligands.split('ENDMDL\n')
    if pose_number < len(ligand_models):
        viewer.addModel(ligand_models[pose_number], 'pdb')
        viewer.setStyle({'model': 2}, {'stick': {}})
    else:
        print(f'Pose number {pose_number} is out of range. Please select a valid pose number.')

# Example usage: dynamically select pose number
pose_number = 0  # Change this to the desired pose number
add_ligand_pose(viewer, ligands, pose_number)

# Zoom to fit all models
viewer.zoomTo()

# Show the viewer
viewer.show()

Vamos a ver la diferencia en RMSD entre los resultados de docking.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Función para calcular el RMSD
def calc_rmsd(coords1, coords2):
    diff = coords1 - coords2
    return np.sqrt(np.mean(np.sum(diff**2, axis=1)))

# Ruta al archivo PDB con varios modelos de docking
docking_pdb = 'redock.pdb'

# Leer coordenadas del archivo de referencia
# Lista para almacenar los valores de RMSD
rmsd_values = []

# Leer el archivo de docking y calcular RMSD para cada modelo
with open(docking_pdb, 'r') as f:
    model_coords = []
    ref = None
    for line in f:
        if line.startswith('MODEL'):
            if model_coords:
                if type(ref)==type(None):
                  ref = np.array(model_coords)
                rmsd = calc_rmsd(ref, np.array(model_coords))
                rmsd_values.append(rmsd)
                model_coords = []
        elif line.startswith('ATOM') and line[13:14] != 'H':
            x = float(line[30:38])
            y = float(line[38:46])
            z = float(line[46:54])
            model_coords.append([x, y, z])
    if model_coords:
        rmsd = calc_rmsd(ref, np.array(model_coords))
        rmsd_values.append(rmsd)

# Generar el gráfico
plt.figure(figsize=(10, 6))
plt.plot(rmsd_values, marker='o', linestyle='-', color='b')
plt.xlabel('Modelo de Docking')
plt.ylabel('RMSD (Å)')
plt.title('RMSD entre los modelos de docking')
plt.grid(True)
plt.show()

Ahora vamos a hacer lo mismo con `diffdock`. Primero instalaremos el programa.

In [ ]:
# @title Instalar diffdock
!pip install ipython-autotime --quiet
import os


if not os.path.exists("/content/DiffDock"):
    %cd /content
    !git clone https://github.com/gcorso/DiffDock.git
    %cd /content/DiffDock
    !git checkout a6c5275 # remove/update for more up to date code

if not os.path.exists("/content/DiffDock/esm"):
    %cd /content/DiffDock
    !git clone https://github.com/facebookresearch/esm
    %cd /content/DiffDock/esm
    !git checkout ca8a710 # remove/update for more up to date code
    !sudo pip install -e .

%cd /content
%env HOME=esm/model_weights
%env PYTHONPATH=$PYTHONPATH:/content/DiffDock/esm
%env diffdock=/content/DiffDock/inference.py

!pip install biopandas --quiet
!pip install biopython --quiet

!pip install torch==2.5
import torch
print(torch.__version__)


!pip install torch_geometric --quiet
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.5.0+cu124.html --quiet
!pip install pyyaml --quiet
!pip install e3nn --quiet
!pip install spyrmsd --quiet

%cd /content

Vamos a definir funciones para usar `diffdock`

In [ ]:
import os
import requests
import time
from random import random

def download_smiles_str(pubchem_id: str, retries:int = 2) -> str:
    """Given a pubchem id, get a smiles string"""
    while True:
        req = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/CID/{pubchem_id}/property/CanonicalSMILES/CSV")
        smiles_url_csv = req.text if req.status_code == 200 else None
        if smiles_url_csv is not None:
            break
        if retries == 0:
            return None
        time.sleep(1+random())
        retries -= 1

    return smiles_url_csv.splitlines()[1].split(',')[1].strip('"').strip("'") if smiles_url_csv is not None else None

In [ ]:
with open("/content/input_protein_ligand.csv", 'w') as out:
    out.write("protein_path,ligand\n")
    out.write(f"/content/rec.pdb,CC1=C(C(=O)N2CCCCC2=N1)CCN3CCC(CC3)C4=NOC5=C4C=CC(=C5)F\n") #Smiles de resperidone

Vamos a usar DiffDock.

In [ ]:
# @title Preparar archivos para Diffdock
%cd /content/DiffDock
!python datasets/esm_embedding_preparation.py --protein_ligand_csv /content/input_protein_ligand.csv --out_file data/prepared_for_esm.fasta
%env HOME=esm/model_weights
%env PYTHONPATH=$PYTHONPATH:/content/DiffDock/esm
!python /content/DiffDock/esm/scripts/extract.py esm2_t33_650M_UR50D data/prepared_for_esm.fasta data/esm2_output --repr_layers 33 --include per_tok --truncation_seq_length 30000

In [ ]:
# @title Usando Diffdock
%cd /content/DiffDock
!python -m inference --protein_ligand_csv /content/input_protein_ligand.csv --out_dir results/user_predictions_small --inference_steps 10 --samples_per_complex 9 --batch_size 1

import re
import pandas as pd
from glob import glob
from shlex import quote
from datetime import datetime
from tqdm.auto import tqdm
from google.colab import files

%cd /content/DiffDock/results/user_predictions_small
results_dirs = glob("./index*")

rows = []
for results_dir in tqdm(results_dirs, desc="runs"):
    results_smiles = re.findall("pdb_+(.+)", results_dir)[0]
    results_sdfs = [os.path.join('/content/DiffDock/results/user_predictions_small', results_dir, f) for f in os.listdir(results_dir) if "confidence" in f and f.endswith(".sdf")]

rank = [int(sdf.split('/')[-1].split('_')[0].strip('rank')) for sdf in results_sdfs]
ordered_rank = np.argsort(rank)

print(ordered_rank)

In [ ]:
# Load protein and reference ligand from PDB files

%cd /content
for result in results_sdfs:
  name = result.split('/')[-1][:5]
  command = f"obabel -isdf '{result}' -opdb -O {name}.pdb"
  os.system(command)

protein_file = 'rec.pdb'
reference_ligand_file = 'lig.pdb'
dock_file="rank1.pdb"

protein = read_pdb(protein_file)
reference = read_pdb(reference_ligand_file)
ligands = read_pdb(dock_file)

# Initialize py3Dmol viewer
viewer = py3Dmol.view(width=800, height=600)

# Add protein to viewer
viewer.addModel(protein, 'pdb')
viewer.setStyle({'model': 0}, {'cartoon': {'color': 'spectrum'}})

# Add reference ligand to viewer
viewer.addModel(reference, 'pdb')
viewer.setStyle({'model': 1}, {'stick': {'color': 'red'}})



viewer.addModel(ligands, 'pdb')
viewer.setStyle({'model': 2}, {'stick': {}})



# Zoom to fit all models
viewer.zoomTo()

# Show the viewer
viewer.show()

## Docking 5 candidatos
Vamos a realizar docking con 5 candidatos clasicamente con `smina` y con `diffdock`. Los farmármacos van a ser risperidona, haloperidol, 5-hydroxytryptamine, aripiprazol y caripracina.

In [ ]:
%cd /content/
ligandos = ['risperidona', 'haloperidol', '5ht', 'aripiprazol', 'caripracina']
smiles = ['CC1=C(C(=O)N2CCCCC2=N1)CCN3CCC(CC3)C4=NOC5=C4C=CC(=C5)F',
          'C1CN(CCC1(C2=CC=C(C=C2)Cl)O)CCCC(=O)C3=CC=C(C=C3)F',
          'C1=CC2=C(C=C1O)C(=CN2)CCN',
          'C1CC(=O)NC2=C1C=CC(=C2)OCCCCN3CCN(CC3)C4=C(C(=CC=C4)Cl)Cl',
          'CN(C)C(=O)NC1CCC(CC1)CCN2CCN(CC2)C3=C(C(=CC=C3)Cl)Cl',
          ]

# Preparamos el input the diffdock
with open("/content/input_protein_ligand.csv", 'w') as out:
    out.write("protein_path,ligand\n")
    for lig in smiles:
      out.write(f"/content/rec.pdb,{lig}\n") #Smiles de resperidone

# Docking con smina


for lig, smi in zip(ligandos, smiles):
    with open(f"{lig}.smi", 'w') as f:
      f.write(smi)
    command = f"obabel -ismi {lig}.smi -opdb -O {lig}.pdb -p --gen3d"
    os.system(command)

    # run smina
    command = f"./smina.static -r rec.pdb -l {lig}.pdb --autobox_ligand lig.pdb --autobox_add 8 --exhaustiveness 16 -o {lig}.pdbqt  --scoring vina > {lig}.out" # vinardo, ad4_scoring, dkoes_fast, dkoes_scoring
    os.system(command)


Vamos a hacer un plot de los resultados de `smina`.

In [ ]:
import re
import numpy as np

def parse_smina_output(file):
    with open(file, "r") as f:
        file_content=f.read()
    # Expresión regular para coincidir con la información de scoring
    score_pattern = re.compile(r'^\d+\s+(-?\d+\.\d+)\s+(\d+\.\d+)\s+(\d+\.\d+)\s*$')
    scores = []
    # Iterar sobre cada línea en el contenido del archivo
    for line in file_content.split('\n'):
        match = score_pattern.match(line)
        if match:
            scores.append(float(match.group(1)))

    return scores


# Diccionario para almacenar los scores
scores = {}

# Iterar sobre los archivos de salida en la carpeta
for lig in ligandos:
      scores[lig] = parse_smina_output(f"{lig}.out")

# Crear el gráfico de barras
fig, ax = plt.subplots()
molecules = list(scores.keys())
all_scores = [scores[molecule] for molecule in molecules]
offset = np.array([-0.4, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.4])

# Crear barras para cada molécula
for i, molecule_scores in enumerate(all_scores):
    ax.bar(offset+i, molecule_scores, 0.1)

ax.set_xlabel('Molécula')
ax.set_ylabel('Score')
ax.set_title('Scores de Docking por Molécula')
ax.set_xticks(range(0,5))
ax.set_xticklabels(scores.keys())

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Ahora vamos a hacer lo mismo con `DiffDock`.

In [ ]:
# @title Correr DiffDock
%cd /content/DiffDock
!python datasets/esm_embedding_preparation.py --protein_ligand_csv /content/input_protein_ligand.csv --out_file data/prepared_for_esm.fasta
%env HOME=esm/model_weights
%env PYTHONPATH=$PYTHONPATH:/content/DiffDock/esm
!python /content/DiffDock/esm/scripts/extract.py esm2_t33_650M_UR50D data/prepared_for_esm.fasta data/esm2_output --repr_layers 33 --include per_tok --truncation_seq_length 30000
%cd /content/DiffDock
!python -m inference --protein_ligand_csv /content/input_protein_ligand.csv --out_dir results/5_ligands --inference_steps 10 --samples_per_complex 10 --batch_size 5

import re
import pandas as pd
from glob import glob
from shlex import quote
from datetime import datetime
from tqdm.auto import tqdm
from google.colab import files

%cd /content/DiffDock/results/5_ligands
results_dirs = glob("./index*")

rows = []
for results_dir in tqdm(results_dirs, desc="runs"):
    # results_pdb_file = "/tmp/pdb/" + re.findall("tmp-pdb-(.+\.pdb)", results_dir)[0]
    results_smiles = re.findall("pdb_+(.+)", results_dir)[0]
    results_sdfs = [os.path.join('/content/DiffDock/results/5_ligands', results_dir, f) for f in os.listdir(results_dir) if "confidence" in f and f.endswith(".sdf")]

rank = [int(sdf.split('/')[-1].split('_')[0].strip('rank')) for sdf in results_sdfs]
ordered_rank = np.argsort(rank)


In [ ]:
%cd /content
for result in results_sdfs:
  name = result.split('/')[-1][:5]
  lig = result.split('/')[-2][:6]
  command = f"obabel -isdf '{result}' -opdb -O {lig}_{name}.pdb"
  os.system(command)

In [ ]:
import shutil
from google.colab import files

# Specify the folder you want to zip
folder_to_zip = '/content'  # Replace with your folder name

# Create a zip file
shutil.make_archive(folder_to_zip, 'zip', folder_to_zip)

# Download the zip file
files.download(f'{folder_to_zip}.zip')